# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

LANGSMITH_API_KEY=****60e8
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
AZURE_OPENAI_PROJECT_ENDPOINT=****ject
AZURE_OPENAI_API_KEY=****U7lG
AZURE_OPENAI_ENDPOINT=****com/
AZURE_OPENAI_API_VERSION=****view
AZURE_OPENAI_DEPLOYMENT=****mini


In [3]:
from langchain.agents import create_agent
from dotenv import load_dotenv
import os
from langchain_openai import AzureChatOpenAI


llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),  # or your deployment
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian",
)

## No Streaming (invoke)

In [4]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why did the scarecrow win an award?

Because he was outstanding in his field!


## values
You have seen this streaming mode in our examples so far. 

In [5]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why did the scarecrow win an award? 

Because he was outstanding in his field!


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [6]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

In a cozy little house where the sun shines bright,  
Lived a family of four, oh what a delight!  
Mom baked cookies, warm and sweet,  
While Dad danced around on his silly feet.  

Sister painted rainbows, colors galore,  
And Brother built castles, oh, what a score!  
They laughed and they played, from morning till night,  
In their world of joy, everything felt right.  

They’d gather for dinner, a feast on the table,  
Sharing stories and giggles, as much as they’re able.  
With each passing moment, their love only grew,  
In this family of laughter, there’s always room for two.  

So here’s to the families, both big and small,  
With love in their hearts, they can conquer it all.  
Through ups and through downs, they stand side by side,  
In the journey of life, they take it in stride!  

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [8]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=llm,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='abd618b6-d788-4856-a69f-73f608873856')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='abd618b6-d788-4856-a69f-73f608873856'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 51, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f97eff32c5', 'id': 'chatcmpl-D15RDnSepJegNsI93A2c43Exurbmw', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': F

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])